## 10 animal classifier model

In [8]:
#NB: Kaggle requires phone verification to use the internet or a GPU. If you haven't done that yet, the cell below will fail
#    This code is only here to check that your internet is enabled. It doesn't do anything else.
#    Here's a help thread on getting your phone number verified: https://www.kaggle.com/product-feedback/135367

import socket,warnings
try:
    socket.setdefaulttimeout(1)
    socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect(('1.1.1.1', 53))
except socket.error as ex: raise Exception("STOP: No internet. Click '>|' in top right and set 'Internet' switch to on")

In [9]:
# It's a good idea to ensure you're running the latest version of any libraries you need.
# `!pip install -Uqq <libraries>` upgrades to the latest version of <libraries>
# NB: You can safely ignore any warnings or errors pip spits out about running as root or incompatibilities
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    !pip install -Uqq fastai

In [10]:
# Skip this cell if you already have duckduckgo_search installed
!pip install -Uqq duckduckgo_search

In [11]:
from duckduckgo_search import ddg_images
from fastcore.all import *
from fastdownload import download_url
from fastai.vision.all import *


def search_images(term, max_images=200): return L(ddg_images(term, max_results=max_images)).itemgot('image')

In [12]:
searches = 'wolf','bird', 'rabbit', 'cat', 'dolphin', 'elephant', 'turtle', 'koala', 'kangaroo', 'lizard'
path = Path('animals')
from time import sleep

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} photo'))
    sleep(10)  # Pause between searches to avoid over-loading server
    download_images(dest, urls=search_images(f'{o} sun photo'))
    sleep(10)
    download_images(dest, urls=search_images(f'{o} shade photo'))
    sleep(10)
    resize_images(path/o, max_size=400, dest=path/o)

UnboundLocalError: ignored

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path)

dls.show_batch(max_n=6)

In [ ]:
fnames = get_image_files(path)

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate, loss_func=CrossEntropyLossFlat())
learn.fine_tune(3)

In [ ]:
correct = 0
for x in fnames:
  animal_pred,_,probs = learn.predict(x)
  #print(f"This is a: {animal_pred}.")
  name_x = str(x).split("/")[1]
  if animal_pred == name_x:
    correct += 1

print(correct/len(fnames)*100)

In [ ]:
animal_pred,_,probs = learn.predict(fnames[0])
print(animal_pred)
